In [ ]:
from utils import *

In [ ]:
units, groups = list_units(write_units_json=True, write_groups_json=True)

In [ ]:
accreds = list_accreds(units, write_accreds_json=True)

In [ ]:
units = compute_units_size(units, accreds)

In [ ]:
users = compute_users_size(accreds, write_users_json=True)

In [ ]:
links = compute_links(accreds, units, users, write_links_json=True)

In [ ]:
write_data(units, users, links)